In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from catboost import Pool, CatBoostClassifier

In [2]:
def read_and_preprocess(filepath):
    
    # initial reading
    data = pd.read_csv(filepath, index_col=0)
    
    # move species from index to the table
    data["species"] = data.index
    
    # create numeric idex
    data.index = range(len(data))
    
    # create categorical feature from Phylum (string)
    data["Phylum_Numeric"] = LabelEncoder().fit_transform(data["Phylum"].tolist())
    
    # remove features we will not use
    data = data.drop(["Phylum", "species", "occurrences", "NoSpecies", 
                      "C_Cnumeric", "SC_Numeric", "MaxD_Numeric", "System_Numeric"], axis=1)
    
    # create features and target dataframes
    features = data.drop(["extinct"], axis=1)
    target = data["extinct"]
    
    # create lists with categorical and continious features' names
    continious_cols = []
    categorical_cols = features.columns.drop(continious_cols).tolist()
    
    # make list of indexes
    categorical_idx = [features.columns.tolist().index(col) for col in categorical_cols]
        
    return features, target, categorical_idx

In [3]:
def model_me(filepath):
    
    # prepare data for modeling
    features, target, cat_idx = read_and_preprocess(filepath)
    
    # create cross-validation instance
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    
    # create holders for scores and feature importances
    cv_scores = []
    feature_importances = []
    
    # loop over different validation splits and save results
    for train_idx, test_idx in cv.split(features, target):
        
        # Gradient boosting model instance
        model = CatBoostClassifier(loss_function="Logloss", random_seed=0)
        
        # Create Pool data classes for train/test
        pool_train = Pool(features.iloc[train_idx, :], target[train_idx], cat_features=cat_idx)
        pool_test = Pool(features.iloc[test_idx, :], target[test_idx], cat_features=cat_idx)
        
        # Train model
        model.fit(pool_train, verbose=False)
        
        # save score from the individual split
        cv_scores.append(roc_auc_score(target[test_idx], model.predict_proba(pool_test)[:, 1]))
        
        # save feature importances from the individual split
        feature_importances.append(model.feature_importances_)
    
    cv_scores = pd.DataFrame(np.array(cv_scores),
                             index=["Split1", "Split2", "Split3", "Split4", "Split5"],
                             columns=["AUC"])
    
    feature_importances = pd.DataFrame(np.array(feature_importances), 
                                       index=["Split1", "Split2", "Split3", "Split4", "Split5"], 
                                       columns=features.columns)
    
    return cv_scores, feature_importances

In [4]:
%%time

# loop over individual Time Intervals
for i in range(1, 5):
    
    # obtain scores and feature importances using cross-validation
    cv, fi = model_me(f"../data/TimeInterval{i}.csv")
    
    cv.to_csv(f"../results/TimeInterval{i}_AUC.csv")
    fi.to_csv(f"../results/TimeInterval{i}_FI.csv")
    
    print(cv.mean())
    print(fi.mean())

AUC    0.677653
dtype: float64
K_Numeric          6.009523
Min_Numeric       11.005016
C_Numeric          7.428270
S_Numeric          7.105958
O_Numeric         10.945989
T_Numeric          7.202962
M_Numeric          5.298124
R_Numeric          6.737482
Re_Numeric         6.216519
MinD_Numeric      21.089964
Phylum_Numeric    10.960192
dtype: float64
AUC    0.760755
dtype: float64
K_Numeric          6.104160
Min_Numeric        8.376890
C_Numeric         14.144111
S_Numeric          8.102869
O_Numeric          6.048638
T_Numeric         10.624248
M_Numeric          5.308389
R_Numeric         10.976457
Re_Numeric         5.290154
MinD_Numeric      18.899386
Phylum_Numeric     6.124696
dtype: float64
AUC    0.749924
dtype: float64
K_Numeric         12.857381
Min_Numeric       16.384291
C_Numeric          5.636839
S_Numeric          9.141847
O_Numeric          9.251088
T_Numeric          6.417911
M_Numeric         12.877255
R_Numeric          4.942675
Re_Numeric         7.505337
MinD_Nume